In [8]:
from collections import defaultdict
from pathlib import Path
import numpy as np
import pandas as pd
from collections import namedtuple

from adversarial_debiasing import AdversarialDebiasing
from load_data import load_data, transform_data


In [9]:
# Load the google analogies training dataset:
analogy_dataset = load_data()
analogy_dataset

[Raw_Datapoint(x1='Athens', x2='Greece', x3='Baghdad', y='Iraq', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Bangkok', y='Thailand', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Beijing', y='China', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Berlin', y='Germany', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Bern', y='Switzerland', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Cairo', y='Egypt', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Canberra', y='Australia', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Hanoi', y='Vietnam', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Havana', y='Cuba', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Helsinki', y='Finland', task='capital-common-cou

In [10]:
# Transform the data such that it includes the embeddings
analogy_dataset = transform_data(analogy_dataset)
analogy_dataset



FileNotFoundError: [Errno 2] No such file or directory: 'data/enwiki_20180420_win10_100d.txt/data'

In [ ]:
# Now we fit a dataset.


model = AdversarialDebiasing()
model.fit(dataset=analogy_dataset)
